In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [6]:
data = pd.read_csv('handwriting.csv')

In [16]:
d = data.iloc[:, 0]
X = data.iloc[:, 1:]

In [ ]:
def image_data()

xes = plt.figure().add_axes((0, 0, 1, 1))
axes.imshow(image_data(X, i), cmap='gray')
plt.show()
